In [1]:
!pip install cassandra-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 26.1 MB/s eta 0:00:0000:0100:01


In [20]:
from cassandra.cluster import Cluster
cluster = Cluster(['node1'], port = 9042)

In [21]:
session = cluster.connect()

## Table Creation

In [22]:
# Create Keyspace
session.execute("""
      CREATE KEYSPACE IF NOT EXISTS dungeons WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 2};
""")

In [23]:
cluster.metadata.keyspaces

{'system_auth': <cassandra.metadata.KeyspaceMetadata at 0x7f852940d610>,
 'system_schema': <cassandra.metadata.KeyspaceMetadata at 0x7f8529443390>,
 'dungeons': <cassandra.metadata.KeyspaceMetadata at 0x7f8528b64a90>,
 'bbdd2': <cassandra.metadata.KeyspaceMetadata at 0x7f8528b64c90>,
 'system_distributed': <cassandra.metadata.KeyspaceMetadata at 0x7f8528b652d0>,
 'system': <cassandra.metadata.KeyspaceMetadata at 0x7f8528b66d10>,
 'system_traces': <cassandra.metadata.KeyspaceMetadata at 0x7f8528b72f90>,
 'system_views': <cassandra.metadata.KeyspaceMetadata at 0x7f8528b73e50>,
 'system_virtual_schema': <cassandra.metadata.KeyspaceMetadata at 0x7f8528b78050>}

In [29]:
session.set_keyspace("dungeons")

In [34]:
print([e for e in session.execute('''DESCRIBE TABLES;''')])

[Row(keyspace_name='dungeons', type='table', name='top_horde')]


In [31]:
session.execute('''CREATE TABLE top_horde (
    country text,
    event_id int,
    email text,
    user_name text,
    monster_id int,
    PRIMARY KEY ((country, event_id), email)
);''')

In [33]:
session.execute('''COPY top_horde (country, event_id, email, user_name, monster_id) FROM 'kill_event.csv' WITH HEADER = TRUE;''')

SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 1:0 no viable alternative at input 'COPY' ([COPY]...)">

In [16]:
session.execute('''DROP TABLE top_horde;''')

## Data Insertion

In [30]:
import csv
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

with open('kill_event.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)  # Using DictReader for easy access by column header

    for row in reader:
        country = row['country']
        event_id = int(row['event_id'])
        email = row['email']
        user_name = row['user_name']
        monster_id = int(row['monster_id'])

        # Insert into user_monster_kills
        insert_query = """
        INSERT INTO user_monster_kills (country, event_id, email, user_name, monster_id)
        VALUES (%s, %s, %s, %s, %s)
        """
        session.execute(insert_query, (country, event_id, email, user_name, monster_id))
        
        # Prepare and execute update for user_kills_counter
        # Since counter updates cannot be batched, we execute them directly
        update_query = """
        UPDATE user_kills_counter SET kills = kills + 1
        WHERE country = %s AND event_id = %s AND email = %s
        """
        session.execute(update_query, (country, event_id, email))


KeyboardInterrupt: 

## Test the query

In [31]:


def get_top_k_users(country, event_id, K):

    # Query to select users based on country and event_id
    query = """
    SELECT email, kills FROM user_kills_counter
    WHERE country = %s AND event_id = %s
    """
    # Execute the query
    rows = session.execute(query, (country, event_id))

    # Convert rows to a list of dicts
    data = [{'email': row.email, 'kills': row.kills} for row in rows]

    # Sort the data by 'kills' in descending order and select top K
    top_k_users = sorted(data, key=lambda x: x['kills'], reverse=True)[:K]

    return top_k_users

# Example usage
country = 'de_DE'
event_id = 10
K = 3
top_k_users = get_top_k_users(country, event_id, K)
print("Top", K, "users with most kills:", top_k_users)


Top 3 users with most kills: []


In [44]:
rows = session.execute('''SELECT country, event_id, email, user_name, COUNT(*) FROM top_horde 
GROUP BY country, event_id, email;''')

In [45]:
data = set(([row.count for row in rows]))
data

{1}